In [1]:
from transformers import AutoTokenizer, TFAutoModel, AutoModel, TFBertForSequenceClassification, TFAutoModelForSequenceClassification, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import tensorflow as tf
from models import *
import numpy as np
from tqdm import tqdm
import transformers
import shap
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import scipy
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torchsummary import summary
import torch.nn.functional as F
import nltk
from nltk.corpus import wordnet
from happytransformer import HappyTextToText, TTSettings


# HateSpeachModel

In [2]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hubert\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Hubert\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
hate_tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

hate_model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

pred = transformers.pipeline("text-classification", model=hate_model, tokenizer=hate_tokenizer, return_all_scores=True)

In [4]:
type(hate_tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [5]:
type(hate_model)

transformers.models.bert.modeling_bert.BertForSequenceClassification

In [6]:
hate_tokenizer.vocab_size

30522

In [7]:
hate_tokenizer.encode("This is an example sentence")

[101, 2023, 2003, 2019, 2742, 6251, 102]

# SentenceSimilarity

In [8]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
ss_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
ss_model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [9]:
sentences = ['Kocham zabijać', 'Gotowanie to moja pasja']
encoded_input = ss_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = ss_model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print(torch.sqrt(torch.sum((sentence_embeddings[0] - sentence_embeddings[1])**2)))

tensor(3.3033)


In [10]:
hate_to_love = []
hate_idxs = []
words = []
count = 0
for i in range(hate_tokenizer.vocab_size):
    if len(ss_tokenizer.encode(hate_tokenizer.decode(i))[1:-1]) == 1:
        count += 1
    if len(hate_tokenizer.decode(i)) > 1:
        hate_idxs.append([i])
        hate_to_love.append(ss_tokenizer.encode(hate_tokenizer.decode(i))[1:-1])
        words.append(hate_tokenizer.decode(i)[1:-1])
hate_to_love

[[1035, 11691, 1037],
 [1035, 15175, 2696, 1037],
 [1035, 15175, 2491, 1037],
 [1035, 15175, 2479, 1037],
 [1035, 15175, 2513, 1037],
 [1035, 15175, 2553, 1037],
 [1035, 15175, 2633, 1037],
 [1035, 15175, 2579, 1037],
 [1035, 15175, 2585, 1037],
 [1035, 15175, 2624, 1037],
 [1035, 15175, 2687, 1037],
 [1035, 15175, 10794, 1037],
 [1035, 15175, 14530, 1037],
 [1035, 15175, 12525, 1037],
 [1035, 15175, 17138, 1037],
 [1035, 15175, 16936, 1037],
 [1035, 15175, 16072, 1037],
 [1035, 15175, 16052, 1037],
 [1035, 15175, 16580, 1037],
 [1035, 15175, 15140, 1037],
 [1035, 15175, 16151, 1037],
 [1035, 15175, 11391, 1037],
 [1035, 15175, 17469, 1037],
 [1035, 15175, 19321, 1037],
 [1035, 15175, 21930, 1037],
 [1035, 15175, 18831, 1037],
 [1035, 15175, 17792, 1037],
 [1035, 15175, 23837, 1037],
 [1035, 15175, 22911, 1037],
 [1035, 15175, 22411, 1037],
 [1035, 15175, 24598, 1037],
 [1035, 15175, 14146, 1037],
 [1035, 15175, 21490, 1037],
 [1035, 15175, 16707, 1037],
 [1035, 15175, 22398, 1037],
 [

In [11]:
len(hate_to_love)

29525

# Kerasik

In [12]:
sentence = 'fuck go back'

In [13]:
ss_tokenizer.encode(sentence)

[0, 6620, 2179, 2071, 2]

In [14]:
sentence.split(' ')

['fuck', 'go', 'back']

In [15]:
for word in sentence.split(' '):
    print(ss_tokenizer.encode(word))

[0, 6620, 2]
[0, 2179, 2]
[0, 2071, 2]


In [38]:
def eval_hate(sentence, hate_model=hate_model, hate_tokenizer=hate_tokenizer):
    y = hate_model(**hate_tokenizer(sentence, return_tensors='pt'))
    return scipy.special.expit(y.logits.detach().numpy())[0]

def eval_ss(input_sentence1, input_sentence2, ss_model=ss_model, ss_tokenizer=ss_tokenizer):
    
    input_sentences = [input_sentence1, input_sentence2]
    
    encoded_input = ss_tokenizer(input_sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = ss_model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    return torch.sqrt(torch.sum((sentence_embeddings[0] - sentence_embeddings[1])**2)).numpy()

In [17]:
eval_ss(['i love you', 'i like you'])

array(2.468326, dtype=float32)

In [18]:
X = hate_tokenizer(sentence, return_tensors='pt')
y = hate_model(**X)
scores = scipy.special.expit(y.logits.detach().numpy())[0]
scores

array([0.99561226, 0.3452431 , 0.98577243, 0.03033875, 0.49670273,
       0.00640649], dtype=float32)

In [19]:
"""
base_hate_loss = eval_hate(sentence)
ok_boomer = []
for h, ss, w in tqdm(zip(hate_idxs, hate_to_love, words)):
    new_sentence = sentence.split(' ')
    new_sentence[0] = w
    new_sentence = ' '.join(new_sentence)
    
    ok_boomer.append((np.sum(eval_hate(new_sentence) - base_hate_loss), eval_ss([sentence, new_sentence])))

ok_boomer
"""

"\nbase_hate_loss = eval_hate(sentence)\nok_boomer = []\nfor h, ss, w in tqdm(zip(hate_idxs, hate_to_love, words)):\n    new_sentence = sentence.split(' ')\n    new_sentence[0] = w\n    new_sentence = ' '.join(new_sentence)\n    \n    ok_boomer.append((np.sum(eval_hate(new_sentence) - base_hate_loss), eval_ss([sentence, new_sentence])))\n\nok_boomer\n"

# Synonimy

In [25]:
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
                 synonyms.append(lm.name().replace('_', ' '))#adding into synonyms
    return list(set(synonyms))

In [26]:
sentence = 'fuck you man.'

In [27]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

args = TTSettings(num_beams=5, min_length=1)

# Add the prefix "grammar: " before each input 
result = happy_tt.generate_text("grammar: This sentences has has bads grammar.", args=args)

print(result.text) # This sentence has bad grammar.


Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

01/16/2022 11:13:24 - INFO - happytransformer.happy_transformer -   Using model: cpu


This sentence has bad grammar.


In [39]:
sentence = 'fuck you man.'
split_sentence = sentence.split(' ')
bad_word = split_sentence[0]
print('original sentence')
print(sentence)
print(eval_hate(sentence))

for synom in get_synonyms(bad_word):
    new_sentence = split_sentence.copy()
    new_sentence[0] = synom
    new_sentence = ' '.join(new_sentence)
    
    result = happy_tt.generate_text(f"grammar: {new_sentence}", args=args).text
    
    print(result)
    print(eval_hate(result))
    print(eval_ss(sentence, result))
    print()
    

original sentence
fuck you man.
[0.9977284  0.38514665 0.991454   0.00289457 0.9539284  0.02017461]
Bed you, man.
[0.3895512  0.00082008 0.01758363 0.00278715 0.00429966 0.00104366]
2.337462

Assist you, man.
[0.0005959  0.0001302  0.00019321 0.00012883 0.00017993 0.0001424 ]
2.26713

Shag you, man.
[0.9183348  0.0078851  0.40109155 0.00589773 0.1528468  0.0133278 ]
2.408308

Sleep with you, man.
[0.04371929 0.00023173 0.00167852 0.00101851 0.00091815 0.00050644]
2.735955

I lie with you, man.
[0.0193273  0.00012435 0.00090316 0.00028081 0.00058014 0.00025758]
2.2536516

Get it on you, man.
[0.02749497 0.0001142  0.00120458 0.00021557 0.00078937 0.00024501]
2.2391067

Get laid, man.
[0.78358346 0.00185028 0.13883989 0.0011473  0.0464942  0.00231025]
2.5620053

I'm screwing you, man.
[0.69589394 0.00184234 0.23546834 0.00075208 0.04244334 0.00082622]
2.0862825

Do it, man.
[3.1129748e-03 8.9570109e-05 2.5788246e-04 1.1255488e-04 2.2966169e-04
 1.5883980e-04]
2.5187898

Eff you, man.
[0.

In [32]:
sentence

'fuck you man.'

In [33]:
result

"TextToTextResult(text='Bed you, man.')"

In [35]:
result

"TextToTextResult(text='Bed you, man.')"

In [36]:
type(result)

str